In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import json

In [2]:
obl1 = pd.read_excel("Мособласть 1 комната.xlsx")
obl2 = pd.read_excel("Мособласть 2 комнаты.xlsx")
obl4 = pd.read_excel("Мособласть 4+ комнаты.xlsx")
obl_st = pd.read_excel("Мособласть cтудия комнаты.xlsx")

# Функция

In [3]:
def procssing(df):
    # Удаляем None
    df = df[df['price'] != '[None]'].copy()
    # Делаем индексы по порядку
    df.index = range(0, df.shape[0])
    
    # Обработка столбца price
    df['price'] = df['price'].apply(lambda x: x.replace("['", '')\
                                 .replace("\\xa0₽']", '')\
                                 .replace(' ', ''))
    # Тип данных price - int
    df['price'] = df['price'].astype('int')
    
    # Обработка столбца address
    df['address'] = df['address'].apply(lambda x: x.replace("['" , '')\
                                               .replace("']", ''))
    
    
    # Обработка столбца apartment_info
    
    # Удаляем ненужные символы
    df['apartment_info'] = df['apartment_info'].apply(lambda x: x.replace('[', '')\
            .replace(']','')\
            .replace("'", '"')\
            .replace('\\xa0м2', '')\
            .replace('\\xa0', '')) 

    # Вместо словаря делаем DataFrame 
    data = pd.DataFrame()
    res = []
    for i in df['apartment_info']:
        data_dict = json.loads(i) 
        data = pd.concat([data, pd.DataFrame([data_dict])], ignore_index=True)
        count = 0
        for keys in data_dict:
            count += 1
        res.append(count)
    
    # Записываем новые столбцы в DataFrame, которые получили на предыдущем шаге
    for i in data.columns:
        df[i] = data[i]
    df['Количество информации в объявлении'] = res
    df.drop('apartment_info', axis = 1, inplace = True)
    
    return df

### Обработка каждого набора данных и заполнение пропущенных значений в столбце 'Комнат'

In [4]:
obl1 = procssing(obl1)
obl1.loc[obl1['Комнат'].isna(), 'Комнат'] = obl1.loc[obl1['Комнат'].isna(), 'Комнат'].fillna(1)

obl2 = procssing(obl2)
obl2.loc[obl2['Комнат'].isna(), 'Комнат'] = obl2.loc[obl2['Комнат'].isna(), 'Комнат'].fillna(2)

obl4 = procssing(obl4)
obl4.loc[obl4['Комнат'].isna(), 'Комнат'] = obl4.loc[obl4['Комнат'].isna(), 'Комнат'].fillna(4)

obl_st = procssing(obl_st)
obl_st['Комнат'] = 0

# Объединение в один набор данных

In [5]:
df = pd.concat([obl1, obl2, obl4, obl_st], ignore_index = True)
print(df.shape)
df.head(3)

(6375, 48)


,price,address,links,Комнат,Площадь,Кухня,Этаж,Количество балконов,Газ,Год постройки,...,Количество подъездов,Санузел,Коммунальные платежи,Теплоснабжение,Залог,Предоплата,Количество информации в объявлении,Энергоснабжение,Прописано,Водоотведение
0,4300000,Московская область городской округ Люберцы раб...,https://domclick.ru/card/sale__flat__1894676156,1,"30,4",8,2,1,есть,1972,...,NaN,NaN,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN
1,5800000,Московская область Лобня Научный городок 3,https://domclick.ru/card/sale__flat__1885409722,1,"31,2","7,3",8,1,есть,1978,...,NaN,NaN,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN
2,6750000,Московская область Ленинский городской округ д...,https://domclick.ru/card/sale__flat__1714472790,1,44,9,2,NaN,NaN,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,14,NaN,NaN,NaN


# Функция отбора и преобразования признаков

In [6]:
def transformation(df):
    info = pd.DataFrame()

    info.index = df.columns
    info['dtypes'] = df.dtypes
    info['NaN'] = df.isna().sum()
    info['count'] = df.count()
    info['NaN%'] = round((info['NaN'] / df.shape[0])*100,2)

    # Отбор признаков в которых < 50% значений NaN
    columns = info[info['NaN%'] < 50].index
    df = df[columns]
    
    # Перевод значений с английского на русский
    df.columns = ['Цена', 'Адрес', 'Ссылка'] + list(df.columns[3:])
    # Изменение порядка признаков
    new_columns = ['Цена', 'Адрес'] + list(df.columns[3:]) + ['Ссылка']
    # Сохранение в наборе данных
    df = df[new_columns]
    
    # Замена , на .
    df['Площадь'] = df['Площадь'].apply(lambda x: x.replace(',', '.'))
    df['Кухня'] = df['Кухня'].apply(lambda x: str(x).replace(',', '.'))
    df['Жилая'] = df['Жилая'].apply(lambda x: str(x).replace(',', '.'))
    
    # Преборбазование
    numeric = ['Комнат', 'Площадь', 'Кухня', 'Этаж', 'Год постройки', 
               'Количество этажей', 'Количество информации в объявлении', 'Жилая']
    for n in numeric:
        df[n] = df[n].astype('float')
    
    
    return df

In [7]:
df = transformation(df)
print(df.shape)
df.head(3)

(6375, 17)


,Цена,Адрес,Комнат,Площадь,Кухня,Этаж,Год постройки,Материал стен,Серия дома,Количество этажей,Тип перекрытий,Лифт,Жилая,Ремонт,Тип сделки,Количество информации в объявлении,Ссылка
0,4300000,Московская область городской округ Люберцы раб...,1.0,30.4,8.0,2.0,1972.0,Кирпичный,v-кирпичный,5.0,Железобетонный,NaN,NaN,NaN,NaN,12.0,https://domclick.ru/card/sale__flat__1894676156
1,5800000,Московская область Лобня Научный городок 3,1.0,31.2,7.3,8.0,1978.0,Кирпичный,v-кирпичный,9.0,Железобетонный,есть,NaN,NaN,NaN,12.0,https://domclick.ru/card/sale__flat__1885409722
2,6750000,Московская область Ленинский городской округ д...,1.0,44.0,9.0,2.0,2015.0,Монолитный,II-01,17.0,Железобетонный,есть,17.0,Евро,Свободная продажа,14.0,https://domclick.ru/card/sale__flat__1714472790


In [8]:
df.isna().sum()

Цена                                     0
Адрес                                    0
Комнат                                   0
Площадь                                  0
Кухня                                  895
Этаж                                     0
Год постройки                          212
Материал стен                          147
Серия дома                            3100
Количество этажей                       49
Тип перекрытий                         852
Лифт                                  2501
Жилая                                 1118
Ремонт                                1490
Тип сделки                             379
Количество информации в объявлении       0
Ссылка                                   0
dtype: int64

In [9]:
df.dtypes

Цена                                    int32
Адрес                                  object
Комнат                                float64
Площадь                               float64
Кухня                                 float64
Этаж                                  float64
Год постройки                         float64
Материал стен                          object
Серия дома                             object
Количество этажей                     float64
Тип перекрытий                         object
Лифт                                   object
Жилая                                 float64
Ремонт                                 object
Тип сделки                             object
Количество информации в объявлении    float64
Ссылка                                 object
dtype: object

In [10]:
df.to_excel('domclick_oblast.xlsx', index = False)